In [ ]:
!pip install autoPyTorch[forecasting]

In [ ]:
from autoPyTorch.api.time_series_forecasting import TimeSeriesForecastingTask
import pandas as pd

In [ ]:
dataset_name = "covid_deaths"

data = pd.read_csv(f"{dataset_name}.csv").set_index('date')
target = 'T1'

targets = data[[target]]

In [ ]:
for forecasting_horizon in [5, 10]:
  y_train = [targets[: -forecasting_horizon]]
  y_test = [targets[-forecasting_horizon:]]

  api = TimeSeriesForecastingTask(seed=0, n_jobs=8, ensemble_size=10, ensemble_nbest=10)

  api.search(
    y_train=y_train,
    optimize_metric='mean_MASE_forecasting',
    n_prediction_steps=forecasting_horizon,
    func_eval_time_limit_secs=500,
    total_walltime_limit=1000
  )

  test_sets = api.dataset.generate_test_seqs()
  y_pred = api.predict(test_sets)

  filepath = f"{dataset_name}_{target}_{forecasting_horizon}.csv"
  predictions_df = pd.read_csv(filepath, index_col=0)
  predictions_df.loc['autoPyTorch'] = y_pred[0]
  predictions_df.to_csv(filepath)